# 分類 - 預測銀行當月風險性

## 標記資料
利用資本適足率(BIS)作為風險分類標準。
由於本國銀行資本適足率全都符合法規標準（皆高於8%），因此以當季資本適足率平均值作為界，當月資本適足率高於平均值的銀行標記為“表現穩健”，低於平均值的銀行標記為“風險承受度稍弱“。

例：2020年第四季本國銀行資本足率平均值為14.89，台灣銀行2020年第四季的BIS為14.95，則台灣銀行在2020年10月、11月、12月的分類標記為“表現穩健”。

## 資料前處理與建模
1.刪除六間銀行（2020年不營業的本國銀行五間、2020年開始營業的銀行一間）<br/>
2.空值補0：銀行可能沒有承辦該項業務，因此沒有資料 <br/>
3.從特徵相關係數矩陣挑出與資本適足率高度相關的特徵作為input <br/>
4.將2008-2017年的資料用SVM建模，預測2018-2020年的分類是否正確

In [18]:
# 載入所需套件
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import svm,metrics
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

import seaborn as sns
import plotly.figure_factory as ff
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [19]:
#匯入資料
df = pd.read_csv('機器學習-本國銀行v3.csv',na_values=['#VALUE!', '#DIV/0!'])
df

,代號,名稱,年月,分類,資本適足率,股本,資產,權益,稅前損益,授信總額,...,逾放比率,備抵呆帳,備抵呆帳覆蓋率,收益總計,費損總計,淨收益,利息收入,利息費用,淨利息總收益比,風險承受性
0,2801,彰銀,2020/12,風險承受度稍弱,14.38,103847,2303458,165064,8312,1553022,...,0.38,18703,3.35,51046,44027,7019,22501,8558,1.986465,0.003047
1,2807,渣打銀行,2020/12,風險承受度稍弱,14.03,29106,715772,46498,2780,307155,...,0.11,4632,15.09,18067,16161,1906,7114,2452,2.445960,0.002663
2,2809,京城銀,2020/12,表現穩健,16.15,11212,321018,46581,6231,196900,...,0.01,2941,117.64,8672,4284,4388,3905,998,0.662489,0.013669
3,2812,台中銀,2020/12,風險承受度稍弱,13.60,41517,728239,57367,4681,485234,...,0.21,6335,6.44,14557,10778,3779,10567,3588,1.846785,0.005189
4,2834,臺企銀,2020/12,風險承受度稍弱,13.42,74886,1791909,98857,5358,1246219,...,0.50,14326,2.34,28192,25850,2342,19672,7803,5.067891,0.001307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5761,5858,臺銀,2008/01,表現穩健,11.87,45000,3128716,239359,645,2012370,...,0.95,13210,0.72,23678,22931,747,5884,5023,1.152610,0.000239
5762,5862,板信銀,2008/01,風險承受度稍弱,8.74,9558,173091,9792,36,134294,...,3.49,1893,0.42,822,776,46,448,281,3.630435,0.000266
5763,5863,瑞興銀,2008/01,表現穩健,13.24,2100,41610,4375,-24,25220,...,1.40,313,0.88,104,129,-25,99,55,-1.760000,-0.000601
5764,5870,花旗台灣,2008/01,表現穩健,22.13,50000,294597,50091,-26,136628,...,3.08,9970,2.46,846,938,-92,510,237,-2.967391,-0.000312


In [20]:
#刪除2020年不營業的銀行
del_list = ['大眾銀','中華銀行','慶豐銀行','澳盛台灣','寶華','樂天銀行']

for i in del_list:
    df.drop(index=(df.loc[(df['名稱']==i)].index),inplace=True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5540 entries, 0 to 5765
Data columns (total 30 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   代號         5540 non-null   object 
 1   名稱         5540 non-null   object 
 2   年月         5540 non-null   object 
 3   分類         5540 non-null   object 
 4   資本適足率      5540 non-null   float64
 5   股本         5540 non-null   int64  
 6   資產         5540 non-null   int64  
 7   權益         5540 non-null   int64  
 8   稅前損益       5540 non-null   int64  
 9   授信總額       5540 non-null   int64  
 10  存款         5540 non-null   int64  
 11  放款         5540 non-null   int64  
 12  權益比率       5540 non-null   float64
 13  資產報酬率(稅前)  5540 non-null   float64
 14  存放比率       5540 non-null   float64
 15  催收款        5540 non-null   int64  
 16  逾期放款       5540 non-null   int64  
 17  甲類逾期放款     5540 non-null   int64  
 18  乙類逾期放款     5540 non-null   int64  
 19  放款(含催收款)   5540 non-null   int64  
 20  逾放比率    

In [23]:
# 檢查空值
df.isnull().sum()

代號           0
名稱           0
年月           0
分類           0
資本適足率        0
股本           0
資產           0
權益           0
稅前損益         0
授信總額         0
存款           0
放款           0
權益比率         0
資產報酬率(稅前)    0
存放比率         0
催收款          0
逾期放款         0
甲類逾期放款       0
乙類逾期放款       0
放款(含催收款)     0
逾放比率         0
備抵呆帳         0
備抵呆帳覆蓋率      1
收益總計         0
費損總計         0
淨收益          0
利息收入         0
利息費用         0
淨利息總收益比      4
風險承受性        0
dtype: int64

In [24]:
# 風險分類計數
pd.DataFrame(df.分類.value_counts()).reset_index()

,index,分類
0,風險承受度稍弱,3102
1,表現穩健,2438


In [25]:
# 風險分類計數長條圖

fig = px.bar(pd.DataFrame(df.分類.value_counts().reset_index()), x='index', y='分類')
fig.show()

## 特徵挑選

In [26]:
# 欄位相關係數矩陣

corrs = df.corr()

corrs_heatmap = ff.create_annotated_heatmap(z=corrs.values,x=list(corrs.columns),y=list(corrs.index), annotation_text=corrs.round(2).values, showscale=True)

corrs_heatmap

In [27]:
# 挑出跟資本適足率強相關的特徵（係數>0.5）：權益比率、存放比率
corrs['資本適足率'].sort_values(ascending=False)

資本適足率        1.000000
權益比率         0.816467
備抵呆帳覆蓋率      0.274714
資產報酬率(稅前)    0.072141
股本           0.070335
權益           0.033574
稅前損益         0.029531
淨收益          0.009967
風險承受性       -0.003525
備抵呆帳        -0.043267
收益總計        -0.045666
費損總計        -0.048821
淨利息總收益比     -0.049758
資產          -0.071920
存款          -0.093361
授信總額        -0.098373
放款          -0.098837
放款(含催收款)    -0.099930
利息收入        -0.106637
利息費用        -0.122586
乙類逾期放款      -0.155507
逾放比率        -0.184726
催收款         -0.227257
逾期放款        -0.227623
甲類逾期放款      -0.232591
存放比率        -0.781973
Name: 資本適足率, dtype: float64

In [97]:
# 分類與權益比的關係

fig = px.box(df, x='分類', y='權益比率')
fig.update_layout(title='2008-2020年銀行各風險分類的權益比率比較')
fig.show()

In [98]:
# 分類與存放比的關係

fig = px.box(df, x='分類', y='存放比率')
fig.update_layout(title='2008-2020年銀行各風險分類的存放比率比較')
fig.show()

## 建立訓練模型

In [29]:
# 重設index
df.reset_index(drop=True,inplace=True) 

# 將分類轉成數值（1:表現穩健/2:風險承受度稍弱）
arr = []
for i in range(len(df)):
    if df['分類'][i] == '表現穩健':
        arr.append(1)
    else: #風險承受度稍弱
        arr.append(2)

df['class'] = arr

In [38]:
# 分割train and test(2008-2017年的資料)
X = df[1296:]

X_train, X_test, y_train, y_test = train_test_split(
    X[['權益比率','存放比率']], X[['class']], test_size=0.3, random_state=0)

In [39]:
# 找最佳參數

def svm_cross_validation(train_x, train_y):    
    from sklearn.model_selection import GridSearchCV   
    from sklearn.svm import SVC    
    model = SVC(kernel='rbf', probability=True)    
    param_grid = {'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1000], 'gamma': [0.001, 0.0001]}    
    grid_search = GridSearchCV(model, param_grid, n_jobs = 8, verbose=1)    
    grid_search.fit(train_x, train_y)    
    best_parameters = grid_search.best_estimator_.get_params()    
    for para, val in list(best_parameters.items()):    
        print(para, val)    
    model = SVC(kernel='rbf', C=best_parameters['C'], gamma=best_parameters['gamma'], probability=True)    
    model.fit(train_x, train_y)    

svm_cross_validation(X_train,y_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
C 100
break_ties False
cache_size 200
class_weight None
coef0 0.0
decision_function_shape ovr
degree 3
gamma 0.0001
kernel rbf
max_iter -1
probability True
random_state None
shrinking True
tol 0.001
verbose False


In [40]:
# SVM建模
svc = svm.SVC(kernel='rbf', C=100, gamma=0.0001)

svc.fit(X_train,y_train)


SVC(C=100, gamma=0.0001)

In [41]:
# 預測測試集

predictions = svc.predict(X_test)

#載入classification_report & confusion_matrix來評估模型好壞

print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))


[[382 175]
 [105 612]]


              precision    recall  f1-score   support

           1       0.78      0.69      0.73       557
           2       0.78      0.85      0.81       717

    accuracy                           0.78      1274
   macro avg       0.78      0.77      0.77      1274
weighted avg       0.78      0.78      0.78      1274



## 預測2018-2020年的風險分類

In [52]:

X_new = df[:1296][['權益比率','存放比率']]
y_new = df[:1296][['class']]

predictions_new = svc.predict(X_new)

In [53]:
# 混淆矩陣評估預測準確度

print(confusion_matrix(y_new,predictions_new))
print('\n')
print(classification_report(y_new,predictions_new))

[[479  76]
 [338 403]]


              precision    recall  f1-score   support

           1       0.59      0.86      0.70       555
           2       0.84      0.54      0.66       741

    accuracy                           0.68      1296
   macro avg       0.71      0.70      0.68      1296
weighted avg       0.73      0.68      0.68      1296



In [87]:
X_new['原分類']=y_new
X_new['預測分類']=predictions_new

In [88]:
# 轉換標籤與重整2018-2020年資料
X_new[['名稱','年月']] = df[:1296][['名稱','年月']]

arr_1 = []
for i in range(len(X_new)):
    if X_new['原分類'][i] == 1:
        arr_1.append('表現穩健')
    else:
        arr_1.append('風險承受度稍弱')

X_new['原分類'] = arr_1

arr_2 = []
for i in range(len(X_new)):
    if X_new['預測分類'][i] == 1:
        arr_2.append('表現穩健')
    else:
        arr_2.append('風險承受度稍弱')

X_new['預測分類'] = arr_2

X_new = X_new[['名稱','年月','權益比率','存放比率','原分類','預測分類']] 
X_new

,名稱,年月,權益比率,存放比率,原分類,預測分類
0,彰銀,2020/12,7.17,77.35,風險承受度稍弱,風險承受度稍弱
1,渣打銀行,2020/12,6.50,46.77,風險承受度稍弱,表現穩健
2,京城銀,2020/12,14.51,82.69,表現穩健,表現穩健
3,台中銀,2020/12,7.88,72.19,風險承受度稍弱,表現穩健
4,臺企銀,2020/12,5.52,81.97,風險承受度稍弱,風險承受度稍弱
...,...,...,...,...,...,...
1291,瑞興銀,2018/01,5.93,69.24,風險承受度稍弱,風險承受度稍弱
1292,花旗台灣,2018/01,12.63,50.31,表現穩健,表現穩健
1293,匯豐台灣,2018/01,6.68,56.82,表現穩健,表現穩健
1294,星展台灣,2018/01,7.23,72.92,風險承受度稍弱,表現穩健


In [92]:
fig = px.box(X_new, x='預測分類', y='權益比率')
fig.update_layout(title='預測2018-2020年銀行風險分類後的權益比率分佈')
fig.show()

In [93]:
fig = px.box(X_new, x='預測分類', y='存放比率')
fig.update_layout(title='預測2018-2020年銀行風險分類後的存放比率分佈')
fig.show()

# 結論

1.從相關係數矩陣中挑選權益比與存放比與資本適足率有高度相關，另外，從盒型圖中觀察到：<br/>
(1)表現穩健銀行的權益比率數值分布較風險承受度較弱的銀行‘高’<br/>
(2)表現穩健銀行的存放比率數值分布較風險承受度較弱的銀行‘廣’

2.以訓練好的模型預測2018-2020年的風險分類，在預測“風險承受度較弱”比”表現穩健“的分類準確（0.84>0.59）